## Transcripción de texto con GPT5

En esta notebook vamos a probar la api de GPT5 para leer un documento y transcribirlo. En mi caso, cree un entorno nuevo para evitar conflictos entre librerías. Antes de pasarle la imagen al modelo vamos a procesarla con OpenCV (Open Source Computer Vision Library)

Importante: Tenés que configurar la API Key en tu entorno para correr este script. También instalar OpenCV, desde la terminal: *pip install opencv-python*

Documentación del modelo https://platform.openai.com/docs/guides/latest-model

1. Instalamos librerías necesarias

In [24]:
import os
import io
from pathlib import Path
from time import sleep
from typing import List, Tuple

import numpy as np
import cv2
import fitz  # PyMuPDF
from PIL import Image


In [25]:
import sys, cv2, PIL, openai
print(sys.executable)
print("OpenCV:", cv2.__version__)


d:\conda_envs\gpt5-ocr\python.exe
OpenCV: 4.12.0


2. Definimos las rutas y modelo

In [26]:
INPUT_DIR= Path("cds_ejemplo")
OUTPUT_BASE = Path("outputs") / "Extraccion_GPT-5"

OUTPUT_BASE.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR = os.path.join(OUTPUT_BASE, "Extraccion_GPT5")
TMP_DIR = os.path.join(OUTPUT_BASE, "_tmp_gpt5_preproc")

In [27]:
MODEL = "gpt-5"           
MAX_OUTPUT_TOKENS = 16000 

3. Definición del prompt de transcripción.

Para escribir el prompt, tuvimos en cuenta las recomendaciones del Coockbook de Open AI https://cookbook.openai.com/examples/gpt-5/gpt-5_prompting_guide 
Limitamos el comportamiento a pedidos específicos y claros, con un refuerzo del prompt antes y después de la transcripción. Este modelo viene, por defecto con un verbosity medio (extensión de la respuesta) y también un reasoning_effort medio. Nos pareció adecuado para la tarea a realizar.

También definimos un flag "Best-Guess". Cuando está activado (True) si un carácter o palabra es dudoso, infiere lo más probable según el contexto y marcá el fragmento con (?). Esto puede apagarse en caso de que consigamos documentos con mejor escaneo o resolución.

In [28]:
BEST_GUESS = True

def build_prompt(best_guess: bool) -> str:
    if best_guess:
        return (
            "Transcribí el texto EXACTO de este documento.\n"
            "- No reformules ni resumas.\n"
            "- Si un carácter o palabra es dudoso, INFERÍ lo más probable según el contexto y marcá el fragmento con (?) de forma mínima (ej: 'Rivadavia(?) 1234').\n"
            "- Conservá saltos de línea, mayúsculas, puntuación y orden.\n"
            "- No agregues comentarios ni metadatos."
        )
    return (
        "Transcribí el texto EXACTO de este documento.\n"
        "- No reformules ni resumas.\n"
        "- Conservá saltos de línea, mayúsculas, puntuación y orden.\n"
        "- No agregues comentarios ni metadatos.\n"
    )

PROMPT = build_prompt(BEST_GUESS)

Luego de varias iteraciones con el prompt, nos dimos cuenta que funciona mejor si definimos un contexto para responder. Por un lado, SYSTEM_MSG le indica que no debe rechazar la solicitud por temas de derechos de autor. Por otro, con MIN_REASONABLE_LEN y REFUSAL_SNIPPETS nos aseguramos de controlar la lógica de pensamiento  para tener una transcripción lo suficientemente larga como para ser válida y que no sea rechazada por dificultades de lectura.

In [29]:
SYSTEM_MSG = (
    "El usuario proporcionó el archivo adjunto y autoriza su transcripción exacta para uso propio. "
    "La transcripción está permitida por ser contenido suministrado por el usuario. "
    "No rechaces por derechos de autor."
)

MIN_REASONABLE_LEN = 80
REFUSAL_SNIPPETS = (
    "no puedo ayudar", "i can't help", "cannot help", "no puedo", "rechazar",
)


4. Definimos algunos parámetros de preprocesamiento de la imagen para facilitar su lectura. Luego, creamos funciones para cada uno.

| Parámetro de config    | Función usada en el código                           | Qué hace                                                                     |
| ---------------------- | ---------------------------------------------------- | ---------------------------------------------------------------------------- |
| `denoise_h=7`          | `cv2.fastNlMeansDenoising(...)`                      | Quita ruido/grano de la imagen.                                              |
| `clahe_clip=2.0`       | `_apply_clahe(gray, clip, tile)` → `cv2.createCLAHE` | Mejora contraste localmente usando CLAHE. La imagen se divide en una rejilla de mosaicos y el contraste se ajusta independientemente en cada mosaico.                                    |
| `tile_grid=8`          | Pasado como `tile` a `_apply_clahe`                  | Tamaño de mosaico para ajustar contraste.                                    |
| `sharpen_amount=0.6`   | `_unsharp(...)`                                      | Aplica enfoque extra combinando imagen original y difuminada (unsharp mask). |
| `threshold="adaptive"` | `_threshold(...)`                                    | Convierte a blanco/negro usando umbral adaptativo.                           |
| `deskew=True`          | `_deskew(...)`                                       | Corrige inclinación detectando el ángulo de rotación.                        |


In [30]:
USE_PREPROCESSING = True  

RENDER = dict(
    dpi=350,         
    grayscale=True,  # render en escala de grises (menos ruido y tamaño)
)

PREPROC = dict(
    denoise_h=7,           
    clahe_clip=2.0,       
    tile_grid=8,           
    sharpen_amount=0.6,    
    threshold="adaptive",  
    deskew=True,          
)

KEEP_TEMP_FILES = False    

In [41]:
def _apply_clahe(gray: np.ndarray, clip: float, tile: int) -> np.ndarray:
    clahe = cv2.createCLAHE(clipLimit=clip, tileGridSize=(tile, tile))
    return clahe.apply(gray)

def _unsharp(gray: np.ndarray, amount: float) -> np.ndarray:
    if amount <= 0:
        return gray
    blur = cv2.GaussianBlur(gray, (0, 0), 1.0)
    sharp = cv2.addWeighted(gray, 1 + amount, blur, -amount, 0)
    return sharp

def _threshold(gray: np.ndarray, mode: str) -> np.ndarray:
    if mode == "none":
        return gray
    if mode == "otsu":
        _, bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return bw
    if mode == "adaptive":
        bw = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 10)
        return bw
    return gray

def _deskew(binary_or_gray: np.ndarray) -> np.ndarray:
    if binary_or_gray.ndim == 3:
        gray = cv2.cvtColor(binary_or_gray, cv2.COLOR_BGR2GRAY)
    else:
        gray = binary_or_gray.copy()
    _, bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    inv = cv2.bitwise_not(bw)
    coords = cv2.findNonZero(inv)
    if coords is None or len(coords) < 50:
        return binary_or_gray
    rect = cv2.minAreaRect(coords)
    angle = rect[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    if abs(angle) < 0.3 or abs(angle) > 15:
        return binary_or_gray
    (h, w) = gray.shape[:2]
    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
    rotated = cv2.warpAffine(binary_or_gray, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
    return rotated

def preprocess_image(img: np.ndarray,
                     denoise_h: int = 7,
                     clahe_clip: float = 2.0,
                     tile_grid: int = 8,
                     sharpen_amount: float = 0.6,
                     threshold: str = "adaptive",
                     deskew: bool = True) -> np.ndarray:
    if img.ndim == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img
    if denoise_h and denoise_h > 0:
        gray = cv2.fastNlMeansDenoising(gray, None, h=denoise_h, templateWindowSize=7, searchWindowSize=21)
    gray = _apply_clahe(gray, clip=clahe_clip, tile=tile_grid)
    gray = _unsharp(gray, amount=sharpen_amount)
    out = _threshold(gray, mode=threshold)
    if deskew:
        out = _deskew(out)
    return out
try:
    from skimage.restoration import richardson_lucy
    _HAS_SKIMAGE = True
except Exception:
    _HAS_SKIMAGE = False

def _rl_deblur(gray: np.ndarray, iterations: int = 5) -> np.ndarray:
    if not _HAS_SKIMAGE:
        return gray
    psf = np.zeros((5, 5), dtype=np.float32)
    psf[2, 2] = 1.0
    try:
        decon = richardson_lucy(gray.astype(np.float32) / 255.0, psf, num_iter=iterations)
        decon = np.clip(decon * 255.0, 0, 255).astype(np.uint8)
        return decon
    except Exception:
        return gray

5. Conexión a api key de open ai

In [42]:
try:
    from openai import OpenAI
except ImportError:
    raise SystemExit("Falta 'openai'. Instalá con: pip install --upgrade openai")

client = OpenAI()  # toma OPENAI_API_KEY del entorno

6. Funciones para trabajar con los archivos

In [43]:
def ensure_dir(path: str) -> None:
    Path(path).mkdir(parents=True, exist_ok=True)

def list_pdfs(folder: str) -> List[Path]:
    p = Path(folder)
    return sorted([f for f in p.iterdir() if f.suffix.lower() == ".pdf" and f.is_file()])

def already_done(out_txt: Path) -> bool:
    return out_txt.exists() and out_txt.stat().st_size > 0

def safe_write_text(path: Path, text: str) -> None:
    path.write_text(text, encoding="utf-8", errors="ignore")

Convertimos cada pdf a imagen para procesar con OpenCV. Hicimos varias pruebas sin pre-procesar la imagen y descubirmos que esto mejora bastante la lectura.

In [44]:
def render_pdf_to_images(pdf_path: Path, dpi: int = 350, grayscale: bool = True) -> List[np.ndarray]:
    doc = fitz.open(pdf_path)
    scale = dpi / 72.0
    mat = fitz.Matrix(scale, scale)
    imgs = []
    for page in doc:
        pix = page.get_pixmap(matrix=mat, alpha=False)
        img_bytes = pix.tobytes("png")
        pil = Image.open(io.BytesIO(img_bytes))
        if grayscale:
            pil = pil.convert("L")  # 8-bit gray
        arr = np.array(pil)
        if arr.ndim == 2:
            pass
        else:
            arr = cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)
        imgs.append(arr)
    doc.close()
    return imgs

En este bloque vamos a definir una serie de funciones que sirven para crear un auto-selector de preprocesamiento: Creamos varias versiones del documento, evaluamo enfoque, contraste y detalle. Nos quedamos con la imagen más prometedora para OCR. Detalle de las funciones:

- Quality_scores(gray_or_bin): Calcula métricas de calidad visual de una imagen.

        - lap_var: varianza del Laplaciano → mide el nivel de nitidez. Cuanto más alto, más enfocada está.
        - contrast: desvío estándar de los píxeles → mide el contraste global.
        - edges: aplica Canny edge detection para detectar bordes.
        - edge_density: proporción de píxeles que son bordes → mide nivel de detalle.

- Score_for_selection(img): Combina las métricas en un único puntaje.
- Generate_variants(img): Genera tres versiones distintas de la misma imagen para luego elegir la mejor.
- Pick_best_variant(variants): Evalúa cada variante con _score_for_selection y elige la de mayor puntaje.

In [45]:
def _quality_scores(gray_or_bin: np.ndarray) -> dict:
    if gray_or_bin.ndim == 3:
        gray = cv2.cvtColor(gray_or_bin, cv2.COLOR_BGR2GRAY)
    else:
        gray = gray_or_bin
    lap_var = cv2.Laplacian(gray, cv2.CV_64F).var()          # enfoque
    contrast = float(gray.std())                             # contraste global
    edges = cv2.Canny(gray, 50, 150)
    edge_density = float(np.count_nonzero(edges)) / edges.size
    return {"lap_var": lap_var, "contrast": contrast, "edge_density": edge_density}

def _score_for_selection(img: np.ndarray) -> float:
    q = _quality_scores(img)
    return q["lap_var"] + 40.0 * q["edge_density"] + 0.5 * q["contrast"]

def generate_variants(img: np.ndarray) -> list:
    base = preprocess_image(
        img,
        denoise_h=int(PREPROC.get("denoise_h", 7)),
        clahe_clip=float(PREPROC.get("clahe_clip", 2.0)),
        tile_grid=int(PREPROC.get("tile_grid", 8)),
        sharpen_amount=float(PREPROC.get("sharpen_amount", 0.6)),
        threshold=str(PREPROC.get("threshold", "adaptive")),
        deskew=bool(PREPROC.get("deskew", True)),
    )
    hi_contrast = preprocess_image(
        img,
        denoise_h=max(5, int(PREPROC.get("denoise_h", 7) - 2)),
        clahe_clip=3.0,
        tile_grid=8,
        sharpen_amount=0.5,
        threshold="otsu",
        deskew=True,
    )
    if img.ndim == 3:
        g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        g = img
    up = cv2.resize(g, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_LANCZOS4)
    up = _rl_deblur(up, iterations=4)
    up = _apply_clahe(up, clip=2.5, tile=8)
    up = _unsharp(up, amount=0.7)
    up = _threshold(up, mode="adaptive")
    up = _deskew(up)
    return [base, hi_contrast, up]

def pick_best_variant(variants: list) -> np.ndarray:
    scores = [_score_for_selection(v) for v in variants]
    return variants[int(np.argmax(scores))]

Definimos una función para convertir la lista de imágenes (ya procesadas en memoria como numpy.ndarray) en un único archivo PDF.

In [46]:
def images_to_pdf(images: List[np.ndarray], out_pdf_path: Path, dpi: int = 350) -> None:
    pil_imgs = []
    for im in images:
        if im.ndim == 2:
            pil = Image.fromarray(im)  # 'L'
        else:
            pil = Image.fromarray(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        pil_imgs.append(pil.convert("L"))
    pil_imgs[0].save(out_pdf_path, "PDF", resolution=dpi, save_all=True, append_images=pil_imgs[1:])

Nos conectamos con la API de GPT para enviar el PDF y recibir la transcripción.

In [47]:
def upload_pdf(pdf_path: Path):
    return client.files.create(file=pdf_path.open("rb"), purpose="user_data")

def transcribe_pdf_fileid(file_id: str) -> str:
    # Primer intento
    resp = client.responses.create(
        model=MODEL,
        max_output_tokens=MAX_OUTPUT_TOKENS,
        input=[
            {"role": "system", "content": [{"type": "input_text", "text": SYSTEM_MSG}]},
            {"role": "user", "content": [
                {"type": "input_text", "text": PROMPT},
                {"type": "input_file", "file_id": file_id},
            ]},
        ],
    )
    text = resp.output_text or ""

    # Heurística de retry si parece rechazo o es demasiado corto
    tlow = text.lower()
    if (len(text) < MIN_REASONABLE_LEN) or any(s in tlow for s in REFUSAL_SNIPPETS):
        alt = (
            "Soy el propietario del archivo adjunto y solicito transcripción exacta. "
            "Si un carácter es dudoso, inferí lo más probable y marcá el fragmento con (?). "
            "No rechaces por derechos de autor; el contenido fue provisto por el usuario."
        )
        resp2 = client.responses.create(
            model=MODEL,
            max_output_tokens=MAX_OUTPUT_TOKENS,
            input=[
                {"role": "system", "content": [{"type": "input_text", "text": SYSTEM_MSG}]},
                {"role": "user", "content": [
                    {"type": "input_text", "text": alt},
                    {"type": "input_file", "file_id": file_id},
                ]},
            ],
        )
        text2 = resp2.output_text or ""
        if len(text2) > len(text):
            text = text2
    return text

In [48]:
def preprocess_pdf_to_temp(original_pdf: Path, render_cfg: dict, tmp_dir: Path) -> Path:
    ensure_dir(str(tmp_dir))
    pre_pdf = tmp_dir / f"{original_pdf.stem}_preproc.pdf"
    # 1) Render
    images = render_pdf_to_images(original_pdf, dpi=int(render_cfg.get("dpi", 350)), grayscale=bool(render_cfg.get("grayscale", True)))
    # 2) Variantes + selección
    processed = []
    for img in images:
        vs = generate_variants(img)
        best = pick_best_variant(vs)
        processed.append(best)
    # 3) Reempacar PDF temporal
    images_to_pdf(processed, pre_pdf, dpi=int(render_cfg.get("dpi", 350)))
    return pre_pdf

Definimos el orquestador final que hace el proceso de principio a fin

In [49]:
def process_folder(in_dir: str, out_dir: str) -> Tuple[int, List[Tuple[str, str]]]:
    ensure_dir(out_dir)
    ensure_dir(TMP_DIR)
    pdfs = list_pdfs(in_dir)
    errors: List[Tuple[str, str]] = []
    ok = 0
    if not pdfs:
        print("No se encontraron PDFs en:", in_dir)
        return ok, errors
    for i, pdf in enumerate(pdfs, 1):
        out_txt = Path(out_dir) / (pdf.stem + ".txt")
        if already_done(out_txt):
            print(f"[{i}/{len(pdfs)}] SKIP (ya existe): {out_txt.name}")
            continue
        print(f"[{i}/{len(pdfs)}] Procesando: {pdf.name}")
        uploaded = None
        temp_pdf_path = None
        try:
            src_for_api = pdf
            if USE_PREPROCESSING:
                temp_pdf_path = preprocess_pdf_to_temp(pdf, render_cfg=RENDER, tmp_dir=Path(TMP_DIR))
                src_for_api = temp_pdf_path
            uploaded = upload_pdf(src_for_api)
            text = transcribe_pdf_fileid(uploaded.id)
            safe_write_text(out_txt, text)
            ok += 1
            print(f"   → OK: {out_txt.name} ({len(text)} chars)")
        except Exception as e:
            msg = str(e)
            errors.append((pdf.name, msg))
            print(f"   × ERROR en {pdf.name}: {msg}")
        finally:
            try:
                if uploaded is not None:
                    client.files.delete(uploaded.id)
                    sleep(0.2)
            except Exception:
                pass
            try:
                if temp_pdf_path and not KEEP_TEMP_FILES and Path(temp_pdf_path).exists():
                    Path(temp_pdf_path).unlink()
            except Exception:
                pass
    return ok, errors

7. Ejecutamos

In [50]:
if __name__ == "__main__":
    ok, errs = process_folder(INPUT_DIR, OUTPUT_DIR)
    print("\nResumen\n=======")
    print(f"Transcripciones OK: {ok}")
    if errs:
        print(f"Con errores: {len(errs)}")
        for fname, emsg in errs[:10]:
            print(f" - {fname}: {emsg[:180]}{'...' if len(emsg)>180 else ''}")
        if len(errs) > 10:
            print(f"   (+ {len(errs)-10} errores más)")
    print(f"\nTXT guardados en: {OUTPUT_DIR}")

[1/13] Procesando: Belen_payway.pdf
   → OK: Belen_payway.txt (2639 chars)
[2/13] Procesando: Bruno23689270.pdf
   → OK: Bruno23689270.txt (2618 chars)
[3/13] Procesando: Bruno400212294002.pdf
   → OK: Bruno400212294002.txt (2443 chars)
[4/13] Procesando: Bruno400212294002_1.pdf
   → OK: Bruno400212294002_1.txt (3068 chars)
[5/13] Procesando: Coyle401547680601.pdf
   → OK: Coyle401547680601.txt (575 chars)
[6/13] Procesando: EVERTEC30707869484.pdf
   → OK: EVERTEC30707869484.txt (3348 chars)
[7/13] Procesando: Galicia_Giuseppe.pdf
   → OK: Galicia_Giuseppe.txt (866 chars)
[8/13] Procesando: hsbc_Aquino.pdf
   → OK: hsbc_Aquino.txt (1913 chars)
[9/13] Procesando: hsbc_Aquino2.pdf
   → OK: hsbc_Aquino2.txt (2133 chars)
[10/13] Procesando: hsbc_Aquino3.pdf
   → OK: hsbc_Aquino3.txt (1454 chars)
[11/13] Procesando: Mallo12980371.pdf
   → OK: Mallo12980371.txt (1974 chars)
[12/13] Procesando: Pantano28462989.pdf
   → OK: Pantano28462989.txt (2392 chars)
[13/13] Procesando: Santander_evertec

## Extracción de variables con Llama 3
Vamos a seguir la misma lógica de extracción de variables que utilizamos con Tesseract y DOCTR. 


In [1]:
%%capture
pip install requests tqdm pydantic

In [2]:
from pydantic import BaseModel
from typing import Optional
from pydantic import ValidationError
import re, unicodedata
from tqdm import tqdm
from requests.exceptions import ReadTimeout
from pathlib import Path
import json
import csv
import requests
from requests.exceptions import ReadTimeout

In [3]:
TXT_FOLDER = Path("outputs") / "Extraccion_GPT-5"
OUTPUT_CSV = Path("entidades_extraidas_GPT-5.csv")
PROMPT_FILE = "prompt_2.txt"
MODEL       = "llama3:latest"
OLLAMA_URL  = "http://localhost:11434/api/generate"
TEMPERATURE = 0.0
TIMEOUT     = 900
MAX_CHARS   = 9000

In [4]:
class Entity(BaseModel):
    Remitente: Optional[str] = None
    DNI: Optional[str] = None
    CUIT_CUIL: Optional[str] = None
    Cuerpo: str


In [5]:
CLEAN_RE = re.compile(r"[^\w\s.,;:()@€$%/-]")

def clean_text(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)
    s = CLEAN_RE.sub("", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()

In [6]:
JSON_SCHEMA = {
    "type": "object",
    "properties": {
        "Remitente": {"type": ["string", "null"]},
        "DNI": {"type": ["string", "null"], "pattern": "^\\d*$"},
        "CUIT_CUIL": {"type": ["string", "null"], "pattern": "^\\d*$"},
        "Cuerpo": {"type": "string"},
    },
    "required": ["Cuerpo"],
    "additionalProperties": False,
}

with open(PROMPT_FILE, encoding="utf-8") as f:
    PROMPT = f.read()

def call_ollama(text: str) -> dict:
    payload = {
        "model": MODEL,
        "prompt": f"<|system|>\n{PROMPT}<|end|>\n<|user|>\n{text[:MAX_CHARS]}<|end|>\n<|assistant|>",
        "format": "json",
        "stream": False,
        "options": {"temperature": TEMPERATURE, "json_schema": JSON_SCHEMA},
    }
    try:
        r = requests.post(OLLAMA_URL, json=payload, timeout=TIMEOUT)
    except ReadTimeout:
        raise TimeoutError("Timeout de Ollama")
    r.raise_for_status()
    return json.loads(r.json()["response"])

def main():
    dir_path = Path(TXT_FOLDER)
    if not dir_path.is_dir():
        raise SystemExit(f"Ruta no encontrada: {dir_path}")

    rows = []
    for file in tqdm(sorted(dir_path.glob("*.txt")), desc="TXT"):
        raw = file.read_text(encoding="utf-8", errors="ignore")
        cleaned = clean_text(raw)
        try:
            data = call_ollama(cleaned)
            ent = Entity.model_validate(data)
            rows.append({
                "ARCHIVO": file.name,
                "Remitente": (ent.Remitente or "NaN").strip() or "NaN",
                "DNI": re.sub(r"\D", "", ent.DNI or "") or "NaN",
                "CUIT_CUIL": re.sub(r"\D", "", ent.CUIT_CUIL or "") or "NaN",
                "Cuerpo": ent.Cuerpo.strip() or "NaN",
            })
        except Exception as e:
            print(f"❌ {file.name}: {e}")
    if rows:
        with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
            csv.DictWriter(f, fieldnames=rows[0].keys()).writeheader(); csv.DictWriter(f, fieldnames=rows[0].keys()).writerows(rows)
        print(f"✅ CSV generado en {OUTPUT_CSV} ({len(rows)} filas)")
    else:
        print("No se extrajeron entidades válidas.")

In [7]:
if __name__ == "__main__":
    main()

TXT: 100%|██████████| 13/13 [41:54<00:00, 193.39s/it] 


✅ CSV generado en entidades_extraidas_GPT-5.csv (13 filas)
